## The Goal of This Project

In this project, your goal is to write a software pipeline to identify the lane boundaries in a video from a front-facing camera on a car. The camera calibration images, test road images, and project videos are available [here](https://github.com/udacity/CarND-Advanced-Lane-Lines).

***

## 0. Required Libraries

In [1]:
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import copy
%matplotlib inline

from moviepy.editor import VideoFileClip
from IPython.display import HTML

***

## 1. Camera Calibration

At first, lens distortion should be corrected using pre-captured checker board image. I use `cv2.findChessboardCorners()` in computer vision library OpenCV to find the corner of the chessboard.
And then, corner position in image coordinate (2D) and world coordinate (3D) is associated.

In [2]:
# get path of checkerboard image
calib_img = glob(".\\camera_cal\\*.jpg")

objpoints = [] # 3D points in real world space
imgpoints = [] # 2D points in image plane

# Arrays to store object points and image points from all the images
objp = np.zeros((6*9, 3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2) #x, y coordinate

for img_path in calib_img:
    img = cv2.imread(img_path)
    
    # convert to Grayscale Image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    
    # if corners are found, add object points, image points
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)
    

***


## 2. Distortion Correction
After 2D-3D association, I pass them to `cv2.undistort()` to apply distortion correction to our dataset.

<small style="color:red">I've separated function used in whole pipeline and debugging functionality for each sections.</small>

In [3]:
def distortion_correction(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)


In [4]:
# create path to store corrected images
directory = os.path.dirname(".\\corrected")
if not os.path.exists(directory):
    os.makedirs(directory)
    
input_images = glob(".\\test_images\\*.jpg")

# calculate camera matrix and distortion coefficient
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

for img_path in input_images:
    img = cv2.imread(img_path)
    # undistort image using parameters calculated above
    dst = distortion_correction(img, mtx, dist)
    fileName = img_path.split("\\")[-1]
    cv2.imwrite(".\\corrected\\" + fileName, dst)

***

## 3. Color & Gradient Threshold

Now that we could get undistorted image, let's start them to lane information extraction.
S channel of HLS color space has good correlation with yellow lane line, so we utilize them combined with vertical edge information extracted from L channel.　An more, I applied thresholding to eliminate shading area to avoid misdetection cast shadow as lane candidate.

This pipeline outputs lane candidate as binarized image.

In [5]:
if not os.path.exists("./binarized"):
    os.makedirs("./binarized")
    

def pipeline(img, s_thresh=(140, 255), sx_thresh=(20, 255), y_thresh=(45, 255)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    kernel = np.ones((3,3),np.uint8)
    # Sobel x
    sobelx = cv2.Sobel(cv2.GaussianBlur(l_channel, (5,5), 0), cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    scaled_sobel = cv2.erode(scaled_sobel, kernel)
    scaled_sobel = cv2.dilate(scaled_sobel, kernel)

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Threshold ___
    y_binary = np.zeros_like(l_channel)
    y_binary[(l_channel >= y_thresh[0]) & (l_channel <= y_thresh[1])] = 1    
    
    # Stack each channel
    color_binary = np.dstack((y_binary, sxbinary, s_binary)) * 255
    return color_binary



In [6]:
corrected_imgs = glob(".\\corrected\\*.jpg")

for corrected_img_path in corrected_imgs:
    img = cv2.imread(corrected_img_path)
    result = pipeline(img)
    fileName = corrected_img_path.split("\\")[-1]
    cv2.imwrite(".\\binarized\\" + fileName, result)

***

## 4. Perspective Transform

Binalized lane candinate will be transformed to bird view image. Now we can see the lane shape correlated with world coordinate system.

In [7]:
if not os.path.exists("./Perspective"):
    os.makedirs("./perspective")

def warper(img, src, dst):
    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image
    return warped


In [8]:
binarized_imgs = glob(".\\binarized\\*.jpg")
#234 691,1070, 691, 604, 442, 674, 442
src = np.array([[248,691], [602,442], [678,442], [1075, 691]], dtype=np.float32)
dst = np.array([[360,719], [360,0], [960,0], [960,719]], dtype=np.float32)

for binarized_img_path in binarized_imgs:
    img = cv2.imread(binarized_img_path)
    result = warper(img, src, dst)
    fileName = binarized_img_path.split("\\")[-1]
    cv2.imwrite(".\\perspective\\" + fileName, result)

***

## 5. Lane Detection

I adopted histgram based algorithm to detect lane information. Binarized image will be vertically summed up to create histgram. As the lane marking is vertical and sequencial, histgram will have peaks at the lane marking. While creating histograms, image regions are horizontally divided, to cope with curved line.

In [9]:
def integrate_binarized_image(img):
    warp_binarized = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    warp_binarized[(img[:,:,0]>128) & ((img[:,:,1]>128) | (img[:,:,2]>128))] = 1
    return warp_binarized

def find_line(img):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))*255
    print(out_img.shape)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return left_lane_inds, right_lane_inds, nonzerox, nonzeroy, left_fit, right_fit

Function `visualize_laneshape()` defined below draws lane candidate to bird view image. This is just used for debugging purpose.

In [10]:

def visualize_laneshape(img, left_fit, right_fit):
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img = np.zeros_like(img)
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    for i in range(len(left_fitx)):
        y = int(ploty[i])
        x = int(left_fitx[i])
        if(y >= 0 and y < img.shape[0] and x >= 0 and x < img.shape[1]):
            #print(left_fitx[i], ploty[i])
            out_img[y, x] = [0, 255, 255]

        x = int(right_fitx[i])
        if(y >= 0 and y < img.shape[0] and x >= 0 and x < img.shape[1]):
            out_img[y, x] = [0, 255, 255]    
    
    return out_img

In [27]:
if not os.path.exists("./estimated"):
    os.makedirs("./estimated")

perspective_imgs = glob(".\\perspective\\*.jpg")

for perspective_img_path in perspective_imgs:
    img = cv2.imread(perspective_img_path)
    binarized = integrate_binarized_image(img)
    left_lane_inds, right_lane_inds, nonzerox, nonzeroy, left_fit, right_fit = find_line(binarized)
    result = visualize_laneshape(img, left_fit, right_fit)
    fileName = perspective_img_path.split("\\")[-1]
    cv2.imwrite(".\\estimated\\" + fileName, result)


(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


***

## 6. Curvature Calculation

Next, polyline is converted to curvature. I scaled bird view image so that the estimated polyline will correlate with actual size.

In [23]:
# Define conversions in x and y from pixels space to meters
ym_per_pix = 60/720 # meters per pixel in y dimension
xm_per_pix = 3.7/584 # meters per pixel in x dimension

def curvature_calculation(left_lane_inds, right_lane_inds, nonzerox, nonzeroy):
    
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    y_eval = np.max(ploty)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    ylen = img.shape[0] * ym_per_pix
    left_offset = left_fit_cr[0]*ylen**2 + left_fit_cr[1]*ylen + left_fit_cr[2] - xm_per_pix * img.shape[1]/2
    right_offset = right_fit_cr[0]*ylen**2 + right_fit_cr[1]*ylen + right_fit_cr[2] - xm_per_pix * img.shape[1]/2
    
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    print('left offset', left_offset, "right offset", right_offset)
    print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    return left_offset, right_offset, left_curverad, right_curverad

In [25]:
if not os.path.exists("./estimated"):
    os.makedirs("./estimated")

perspective_imgs = glob(".\\perspective\\*.jpg")

for perspective_img_path in perspective_imgs:
    img = cv2.imread(perspective_img_path)
    warp_binarized = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    warp_binarized[(img[:,:,0]>128) & ((img[:,:,1]>128) | (img[:,:,2]>128))] = 1
    left_lane_inds, right_lane_inds, nonzerox, nonzeroy, left_fit, right_fit = find_line(warp_binarized)
    result = curvature_calculation(left_lane_inds, right_lane_inds, nonzerox, nonzeroy)
    fileName = perspective_img_path.split("\\")[-1]
    cv2.imwrite(".\\estimated\\" + fileName, result)

(720, 1280, 3)
left offset -1.80441908099 right offset 1.95413872681
3320.91096242 m 12737.9209656 m
(720, 1280, 3)
left offset -1.76273377399 right offset 1.97540391419
6880.10980221 m 169360.452706 m
(720, 1280, 3)
left offset -1.62207230161 right offset 2.18137743827
1205.71623955 m 1085.30691214 m
(720, 1280, 3)
left offset -1.40391244725 right offset 2.3613948333
927.604311675 m 1309.97365696 m
(720, 1280, 3)
left offset -1.63482629099 right offset 2.11415613505
1029.30431061 m 1052.40916961 m
(720, 1280, 3)
left offset -1.64394158413 right offset 2.30900290686
1026.16390373 m 807.96626651 m
(720, 1280, 3)
left offset -1.92316215741 right offset 2.11539433903
883.129539898 m 964.240003872 m
(720, 1280, 3)
left offset -1.65826433527 right offset 2.26228165055
849.506416496 m 1514.75510398 m


## 7. Drawing
Finally, estimated road region is overlayed onto source video data.

In [19]:
def overlay_result(orgimg, warpimg, lineL, lineR, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warpimg).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    left_fit = lineL.ave_fit
    right_fit = lineR.ave_fit
    offset = lineL.offset_x + 1.85 
    curvature = (lineL.ave_curvature + lineR.ave_curvature) / 2.0

    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    warpimg[:,:,0] = np.zeros_like(warp_zero[:,:,0])
    cv2.fillPoly(warpimg, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(warpimg, Minv, (img.shape[1], img.shape[0]))
    # Combine the result with the original image
    result = cv2.addWeighted(orgimg, 1, newwarp, 0.3, 0)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    data_curve = ('curvature[m]: ' + "{0:.1f}".format(curvature))
    data_offset = ('offset[m]: ' + "{0:.2f}".format(offset))
    cv2.putText(result,data_curve,(50,50), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result,data_offset,(50,90), font, 1,(255,255,255),2,cv2.LINE_AA)

    return result, pts_left, pts_right

***

## 8. Against Dataset
Now I'll apply my implementation to video sequence. In order to achieve stable result, I adopted some functionality to ignore the false result. Which can be detected by the estimated road width.

In [15]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        #polynomial coefficients averaged over the last n iterations
        self.ave_fit = np.zeros(3)  
        #polynomial coefficients for the most recent fit
        self.current_fit = np.zeros(3)  
        #radius of curvature of the line in some units
        self.curvature = np.zeros(3)
        #radius of curvature averaged over the last n iterations
        self.ave_curvature = np.zeros(3)
        #distance in meters of vehicle center from the line
        self.offset_x = None 
        #x values for detected line pixels
        self.pts = None
        # number of sequencial sucess
        self.num_success = 0
        # initialized?
        self.is_first = True
        
    def update(self, road_width_near, road_width_far):
        if road_width_near > 3.0 and road_width_near < 4.5 and road_width_far > 3.0 and road_width_far < 4.5:
            #print("success")
            self.detected = True
            self.num_success += 1
        else:
            #print("fail")
            self.detected = False
            self.num_success = 0
        if self.num_success == 0:
            pass
        else:
            if self.is_first == True:
                self.is_first = False
                self.ave_fit = self.current_fit
                self.ave_curvature = self.curvature
            else:
                self.ave_fit = self.ave_fit * (0.8) + self.current_fit *(0.2)
                self.ave_curvature = self.ave_curvature * (0.8) + self.curvature * (0.2)

In [16]:

src = np.array([[248,691], [602,442], [678,442], [1075, 691]], dtype=np.float32)
dst = np.array([[360,719], [360,0], [960,0], [960,719]], dtype=np.float32)
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)


def process_image(img, lineL, lineR):
    corrected = distortion_correction(img, mtx, dist)
    color_bin = pipeline(corrected, s_thresh=(120, 255), sx_thresh=(20, 100))
    cv2.imwrite(".\\color_bin.jpg", color_bin)
    warped = warper(color_bin, src, dst)
    mono_warped = integrate_binarized_image(warped)
    left_lane_inds, right_lane_inds, nonzerox, nonzeroy, left_fit, right_fit = find_line(mono_warped)
    #result = visualize_laneshape(warped, left_fit, right_fit)
    offsetL, offsetR, curvatureL, curvatureR = curvature_calculation( \
                                                                     left_lane_inds, right_lane_inds,\
                                                                     nonzerox, nonzeroy)
    lineL.curvature = curvatureL
    lineR.curvature = curvatureR
    road_width_near = offsetR - offsetL
    lineL.offset_x = offsetL
    lineR.offset_x = offsetR
    lineL.current_fit = left_fit
    lineR.current_fit = right_fit
    road_width_far = (lineR.current_fit[2] - lineL.current_fit[2]) * xm_per_pix
    lineL.update(road_width_near, road_width_far)
    lineR.update(road_width_near, road_width_far)
    result, left_pts, right_pts = overlay_result(corrected, warped.astype(np.uint8), lineL, lineR, Minv)
    #print(left_pts[0])
    #lineL,pts = left_pts
    #lineR.pts = right_pts

    return result

### Test per image
this cell should be deleted

In [20]:
#img = cv2.imread(".\\test_images\\test3.jpg")
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
lineL = Line()
lineR = Line()
#result = process_image(img, lineL, lineR)
#plt.imshow(result)

if not os.path.exists("./overlayed"):
    os.makedirs("./overlayed")

overlay_imgs = glob(".\\test_images\\*.jpg")

for overlay_img_path in overlay_imgs:
    img = cv2.imread(overlay_img_path)
    result = process_image(img, lineL, lineR)
    fileName = overlay_img_path.split("\\")[-1]
    cv2.imwrite(".\\overlayed\\" + fileName, result)

(720, 1280, 3)
left offset -1.80290105318 right offset 1.94967180682
3324.62786459 m 11389.7153136 m
(720, 1280, 3)
left offset -1.76297682598 right offset 1.97420345587
6899.62768442 m 86643.0066962 m
(720, 1280, 3)
left offset -1.68736782167 right offset 2.18982441853
4200.03333312 m 1057.61403324 m
(720, 1280, 3)
left offset -1.40667247961 right offset 2.35300485604
922.328498525 m 1289.64959905 m
(720, 1280, 3)
left offset -1.63035587979 right offset 2.11601566347
1018.24519736 m 1049.81148529 m
(720, 1280, 3)
left offset -1.54589505178 right offset 2.29787440297
2211.35807773 m 844.633246328 m
(720, 1280, 3)
left offset -1.92232303568 right offset 2.11829057853
855.399641396 m 962.671031574 m
(720, 1280, 3)
left offset -1.67370901035 right offset 2.2630521344
1527.38507844 m 1509.49457609 m


## Apply to video sequence
Now all functions are ready to be evaluated. I use `moviepy` module to process input movie to my function.

In [18]:
white_output = 'output_images/output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip(".\\project_video.mp4")
lineL = Line()
lineR = Line()

def my_process_image(clip, lineL, lineR):
    def new_process_image(image):
        return process_image(image, lineL, lineR)
    return clip.fl_image(new_process_image)

white_clip = clip1.fx(my_process_image, lineL, lineR) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

(720, 1280, 3)
left offset -1.43901174449 right offset 2.43365913289
995.620155074 m 1789.58384623 m
[MoviePy] >>>> Building video output_images/output.mp4
[MoviePy] Writing video output_images/output.mp4


  0%|                                                                                 | 0/1261 [00:00<?, ?it/s]

(720, 1280, 3)
left offset -1.43901174449 right offset 2.43365913289
995.620155074 m 1789.58384623 m


  0%|                                                                         | 1/1261 [00:00<06:51,  3.06it/s]

(720, 1280, 3)
left offset -1.42923237719 right offset 2.41992421029
978.709332636 m 1703.58312278 m


...

(720, 1280, 3)
left offset -1.37870016629 right offset 2.47659349134
59516.0887163 m 3875.88560533 m


100%|█████████████████████████████████████████████████████████████████████▉| 1259/1261 [06:00<00:00,  3.64it/s]

(720, 1280, 3)
left offset -1.35842129093 right offset 2.49150597141
6431.27451196 m 3298.81649986 m


100%|█████████████████████████████████████████████████████████████████████▉| 1260/1261 [06:01<00:00,  3.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/output.mp4 

Wall time: 6min 2s
